<a href="https://colab.research.google.com/github/tonish/ndwi_exe/blob/main/q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#get all missing packages
#get data

!git clone https://github.com/tonish/ndwi_exe.git
%cd ndwi_exe
!mkdir -p  v1.1/data/flood_events/HandLabeled/S2Hand
!mkdir -p  v1.1/data/flood_events/HandLabeled/LabelHand
!mkdir -p  v1.1/splits

!gsutil -m rsync -r gs://sen1floods11/v1.1/splits ./v1.1/splits
!gsutil cp -r gs://sen1floods11/v1.1/data/flood_events/HandLabeled/S2Hand ./v1.1/data/flood_events/HandLabeled
!gsutil cp -r gs://sen1floods11/v1.1/data/flood_events/HandLabeled/LabelHand ./v1.1/data/flood_events/HandLabeled
!pip install segmentation_models_pytorch lightning torchgeo


In [ ]:
%cd ndwi_exe
from q4_baseline import test_thresh
from q4_xgb import q4_xgb
from UNET.train import train
import torch
import numpy as np
import matplotlib.pyplot as plt
from UNET.dataset import load_imgs
import skimage
import os
import UNET.model
import segmentation_models_pytorch as smp
from collections import OrderedDict
from UNET import config
from UNET.utils import *

In [ ]:
#test the baseline model using only ndwi and threshold = 0
test_IOU_thresh0,baseline_predictions = test_thresh(thresh = 0)
save_predictions('predictions',np.array(baseline_predictions),test_IOU_thresh0,'baseline')

In [ ]:
# train a simple random forest to on train and test
xgb_IOU, xgb_predictions = q4_xgb()
save_predictions('predictions',np.array(xgb_predictions),xgb_IOU,'xgb')

In [ ]:
#train Unet on training split and test
#  ***start a tfboard session to track training***

'''uncomment this for training without a checkpoint'''
# trainer, s2_model, s2DM = train()
# trainer.fit(s2_model, s2DM)

'''uncomment this if a checkpoint exist'''
checkpoint = ''
_trainer, _s2_model, s2DM = train()
s2_model = load_lightning_checkpoint(checkpoint=checkpoint)

s2DM.setup('test')
IOU_UNET,UNET_predictions = check_IOU(s2DM.test_dataloader(), s2_model, device='cuda')
save_predictions('predictions',np.array(UNET_predictions),IOU_UNET,'unet')

In [ ]:
# load predictions and plot example
imgs,labels = load_imgs(config.TEST_PATH)
baselline_predictions = np.load('predictions/baseline_predictions.npy')
xgb_predictions = np.load('predictions/xgb_predictions.npy')
UNET_predictions = np.load('predictions/unet_predictions.npy')
plot_random_test(imgs,labels,baselline_predictions,xgb_predictions,UNET_predictions)